In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from os import walk

import time

%matplotlib inline

In [2]:
pathBase = 'I://Chuan//Documents//MyData//NormalCAG//CoroBase//Arrangement//'
#pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
    
f = []
f_LCA_Cranial_RAO = []
mypath_LCA_Cranial_RAO = pathBase + 'LCA_Cranial_RAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_RAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_RAO.append(mypath_LCA_Cranial_RAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Cranial_RAO'] = f_LCA_Cranial_RAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO

In [3]:
theWhole['LCA_Cranial_LAO'][1].split('.')[-1]

'npy'

In [4]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

22.09092354774475


### 2D to 1D

In [5]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

## Support Vector Classification

In [6]:
x_train_list = []
y_train_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_train_list.append(x_item)
        y_train_list.append(y)

### Training

In [7]:
start = time.time()

X_train = np.array(x_train_list)
y_train = np.array(y_train_list)

# linear kernel
clf = svm.SVC(kernel='linear', decision_function_shape='ovo')
clf.fit(X_train, y_train)

end = time.time()

In [8]:
print(end - start)

119.35802388191223


### Prediction

In [9]:
print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =  [[ 6.20497989e-06  5.27256222e-06  5.43508992e-06 ... -1.02530880e-05
  -1.03174245e-05 -1.06180418e-05]
 [ 3.34567551e-06  2.61397966e-06  2.30155349e-06 ...  1.65413816e-06
   1.63877012e-06  1.94331925e-06]
 [ 1.06673555e-05  1.15503883e-05  1.26264788e-05 ... -2.24476929e-06
  -4.72183913e-07  1.36084921e-06]
 ...
 [-4.75027045e-06 -5.24507958e-06 -5.40273902e-06 ... -4.21538300e-06
  -3.24788572e-06 -2.38320768e-06]
 [-3.83151948e-06 -3.63989352e-06 -3.27653052e-06 ... -5.14394412e-07
  -3.97133100e-07 -3.74053179e-07]
 [-2.78960215e-08  2.34590311e-06  3.51443563e-06 ... -5.88015587e-06
  -6.62432501e-06 -7.91444785e-06]]
b =  [-2.46858202 -1.3445861  -5.32148933 -4.81807765 -1.04084266 -1.2416585
  3.66919523  1.94824251 -1.34188183  3.69622951 -1.38301444 -1.35464201
 -0.71447861 -2.44061491 -3.37349772 -2.32843434  2.91425698  0.10800616
 -7.73938285 -5.30181619 -0.34588569]
Indices of support vectors =  [  1   3   4   5   7   9  10  11  12  16  17  19  20  22  26  27  28

In [10]:
pathBase = 'I://Chuan//Documents//MyData//NormalCAG//CoroBase//Arrangement//test//'
#pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//test//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
    
f = []
f_LCA_Cranial_RAO = []
mypath_LCA_Cranial_RAO = pathBase + 'LCA_Cranial_RAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_RAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_RAO.append(mypath_LCA_Cranial_RAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Cranial_RAO'] = f_LCA_Cranial_RAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO

In [11]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

4.024064540863037


### 2D to 1D

In [12]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

In [13]:
x_test_list = []
y_test_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_test_list.append(x_item)
        y_test_list.append(y)

In [14]:
X_test = np.array(x_test_list)
y_test = np.array(y_test_list)

## Performance Measures

In [15]:
y_predict = clf.predict(X_test)
y_test = y_test

In [16]:
y_predict

array(['LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Spider', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial_LAO', 'RCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial_RAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial_RAO',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial_RAO', 'LCA_Cranial_LAO', 'LCA_Cranial',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial', 'LCA_Spider', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial_

In [17]:
y_test

array(['LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', '

### Confusion Matrix (混淆矩阵)

In [18]:
cm = np.array(confusion_matrix(y_test, y_predict, labels=['LCA_Caudal', 'LCA_Cranial', 
                                                          'LCA_Cranial_LAO', 'LCA_Cranial_RAO', 
                                                          'LCA_Spider', 
                                                          'RCA_Cranial', 'RCA_LAO']))
confusion = pd.DataFrame(cm, index=['LCA_Caudal', 'LCA_Cranial', 
                                    'LCA_Cranial_LAO', 'LCA_Cranial_RAO', 'LCA_Spider',
                                    'RCA_Cranial', 'RCA_LAO'],
                        columns=['Predictive LCA_Caudal', 'Predictive LCA_Cranial',
                                 'Predictive LCA_Cranial_LAO', 'Predictive LCA_Cranial_RAO',
                                 'Predictive LCA_Spider',
                                    'Predictive RCA_Cranial', 'Predictive RCA_LAO'])
confusion.to_csv('ConfusionMatrix.csv')
confusion

,Predictive LCA_Caudal,Predictive LCA_Cranial,Predictive LCA_Cranial_LAO,Predictive LCA_Cranial_RAO,Predictive LCA_Spider,Predictive RCA_Cranial,Predictive RCA_LAO
LCA_Caudal,19,0,0,0,1,0,0
LCA_Cranial,0,18,3,2,0,1,0
LCA_Cranial_LAO,0,7,11,1,1,0,0
LCA_Cranial_RAO,0,5,0,16,0,0,0
LCA_Spider,0,0,0,0,18,1,0
RCA_Cranial,0,2,1,0,0,15,2
RCA_LAO,0,0,0,0,0,0,11


In [19]:
print(classification_report(y_test, y_predict))

                 precision    recall  f1-score   support

     LCA_Caudal       1.00      0.95      0.97        20
    LCA_Cranial       0.56      0.75      0.64        24
LCA_Cranial_LAO       0.73      0.55      0.63        20
LCA_Cranial_RAO       0.84      0.76      0.80        21
     LCA_Spider       0.90      0.95      0.92        19
    RCA_Cranial       0.88      0.75      0.81        20
        RCA_LAO       0.85      1.00      0.92        11

      micro avg       0.80      0.80      0.80       135
      macro avg       0.82      0.82      0.81       135
   weighted avg       0.81      0.80      0.80       135



## Save the Model

In [20]:
c = joblib.dump(clf, 'trained_model.m')
print(c)

['trained_model.m']


## Load the Model

In [21]:
d = joblib.load('trained_model.m')
print(d)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
